In [ ]:
import os
import sys
from datetime import datetime
sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve

from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, LogScale, Axis, Lines, Figure, Scatter
from random import randint    

from numpy import random as np_random
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_json as pd_read_json

from ipywidgets import HBox, VBox

In [ ]:
%matplotlib notebook

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

## Create an account on the HUGS platform

To use the HUGS service an account must be created using the Acquire authentication service. Please follow the instructions below to create your account.

In [ ]:
from Acquire.Client import User
from getpass import getpass

Enter a username and password using the cells below

In [ ]:
username = test
password = test123

In [ ]:
result = User.register(username=username, password=password,
                       identity_url="https://google-cloud.acquire-aaai.com/t/identity")

## OTP QR code

Run the cell below to create a QR code to scan using an authenticator app such as AndOTP or Google Authenticator

In [ ]:
result["qrcode"]

## Login

You are now ready to login to the HUGS platform

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
user = User(username="gareth", identity_url=F"{base_url}/identity")
response = user.request_login()

Check that our login has been successful

In [ ]:
user.wait_for_login()

## TRANSCOM users won't have to upload any data, that'll all be on the platform already

## Search interface - ipywidgets from HUGS.Interface

### Hide all the download keys etc from the user and just give a table of things to plot

### Select the things to plot, return VBox's etc from plotting functions?